# Notebook focused towards fetching funding rate across CEX and save the data to .csv

- [Binance API](https://developers.binance.com/docs/derivatives/usds-margined-futures/market-data/rest-api/Get-Funding-Rate-History)
- [ByBit API](https://bybit-exchange.github.io/docs/api-explorer/v5/market/history-fund-rate)
- [OKX API](https://www.okx.com/docs-v5/en/?python#public-data-rest-api-get-funding-rate-history)
- [Deribit API](https://docs.deribit.com/#public-get_funding_rate_history)
- [Bitget API](https://www.bitget.com/api-doc/contract/market/Get-History-Funding-Rate)